In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
from PIL import Image

# Open the image
img = Image.open('/Users/wwang/Desktop/work/ultrasound breast classification/train/benign/benign (108)-sharpened-sharpened.png')

# Get the image's size (width, height)
width, height = img.size
print(f"The image size is: {width}x{height}")


The image size is: 224x224


In [5]:
import numpy as np
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.models import * 
from tensorflow.keras.preprocessing import image

In [6]:
train_path= '/Users/wwang/Desktop/work/ultrasound breast classification/train'
test_path='/Users/wwang/Desktop/work/ultrasound breast classification/val'

In [7]:
train_datagen = image.ImageDataGenerator(
    rotation_range=15,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    width_shift_range=0.1,
    height_shift_range=0.1
)
val_datagen= image.ImageDataGenerator(    rotation_range=15,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    width_shift_range=0.1,
    height_shift_range=0.1)

In [8]:
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size = (112,112),
    batch_size = 4,
    class_mode = 'binary')
validation_generator = val_datagen.flow_from_directory(
    test_path,
    target_size = (112,112),
    batch_size = 4,
    shuffle=True,
    class_mode = 'binary')

Found 8116 images belonging to 2 classes.
Found 900 images belonging to 2 classes.


In [9]:
base_model = tf.keras.applications.EfficientNetB3(weights='imagenet', input_shape=(224,224,3), include_top=False)

for layer in base_model.layers:
    layer.trainable=False
model = Sequential()
model.add(base_model)
model.add(GaussianNoise(0.25))
model.add(GlobalAveragePooling2D())
model.add(Dense(256,activation='relu'))
model.add(BatchNormalization())
model.add(GaussianNoise(0.25))
model.add(Dropout(0.25))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb3 (Functional)     │ ?                      │    10,783,535 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gaussian_noise (GaussianNoise)  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ ?                      │   0 (unbuilt) │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gaussian_noise_1                │ ?                      │   0 (unbuilt) │
│ (GaussianNoise)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,783,535 (41.14 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 10,783,535 (41.14 MB)

In [10]:
# model.compile(loss='binary_crossentropy',
#               optimizer='adam',
#               metrics=['accuracy','AUC','Precision','Recall'])

In [27]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import time
from sklearn.metrics import classification_report
# es=EarlyStopping(patience=3,monitor='val_loss')
# filepath='best_model.keras'
# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
# print("Processing without padding:")

# # Train the model with reduced steps_per_epoch (e.g., 10 batches per epoch)
# history = model.fit(
#     train_generator,
#     epochs=3,  # Shorten epochs to just 3 for quicker testing
#     validation_data=validation_generator,
#     steps_per_epoch=10,  # Reduce the number of steps (batches) per epoch
#     callbacks=[checkpoint, es]
# )

def train_and_evaluate(target_size, model_name):
    start_time = time.time()  # Start timer
    
    # Create image generators for the given target_size
    train_datagen = image.ImageDataGenerator(
        rotation_range=15,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        width_shift_range=0.1,
        height_shift_range=0.1
    )
    
    val_datagen = image.ImageDataGenerator(
        rotation_range=15,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        width_shift_range=0.1,
        height_shift_range=0.1
    )
    
    # Train and validation generators with the current target size
    train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=target_size,
        batch_size=4,
        class_mode='binary'
    )
    
    validation_generator = val_datagen.flow_from_directory(
        test_path,
        target_size=target_size,
        batch_size=4,
        shuffle=True,
        class_mode='binary'
    )
    
    # Load and build the model
    base_model = tf.keras.applications.EfficientNetB3(weights='imagenet', input_shape=(target_size[0], target_size[1], 3), include_top=False)
    for layer in base_model.layers:
        layer.trainable = False

    model = Sequential()
    model.add(base_model)
    model.add(GaussianNoise(0.25))
    model.add(GlobalAveragePooling2D())
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(GaussianNoise(0.25))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))
    
    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'AUC', 'Precision', 'Recall'])
    
    # Set up callbacks
    es = EarlyStopping(patience=3, monitor='val_loss')
    filepath = f'{model_name}_best_model.keras'
    checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
    
    # Train the model
    history = model.fit(
        train_generator,
        epochs=10,
        validation_data=validation_generator,
        steps_per_epoch=50,
        callbacks=[checkpoint, es]
    )
    
    # End timer and calculate time taken
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Time taken for model {model_name}: {elapsed_time:.2f} seconds")
    
    # Evaluate the model on the validation set
    evaluation = model.evaluate(validation_generator)
    print(f"Evaluation results for {model_name}:")
    print(f"Loss: {evaluation[0]}, Accuracy: {evaluation[1]}, AUC: {evaluation[2]}, Precision: {evaluation[3]}, Recall: {evaluation[4]}")
    
    return evaluation, elapsed_time

results_224, time_224 = train_and_evaluate((224, 224), "model_224")
results_128, time_128 = train_and_evaluate((128, 128), "model_128")
results_64, time_64 = train_and_evaluate((64, 64), "model_64")
results_243, time_243 = train_and_evaluate((243, 243), "model_243")
results_162, time_162 = train_and_evaluate((162, 162), "model_162")
results_81, time_81 = train_and_evaluate((81, 81), "model_81")






Found 8116 images belonging to 2 classes.
Found 900 images belonging to 2 classes.
Epoch 1/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - AUC: 0.5500 - Precision: 0.5364 - Recall: 0.5320 - accuracy: 0.5324 - loss: 1.0642
Epoch 1: val_accuracy improved from -inf to 0.69333, saving model to model_224_best_model.keras
50/50 ━━━━━━━━━━━━━━━━━━━━ 37s 616ms/step - AUC: 0.5519 - Precision: 0.5380 - Recall: 0.5333 - accuracy: 0.5340 - loss: 1.0614 - val_AUC: 0.7757 - val_Precision: 0.6442 - val_Recall: 0.6925 - val_accuracy: 0.6933 - val_loss: 0.5647
Epoch 2/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - AUC: 0.7154 - Precision: 0.7074 - Recall: 0.6179 - accuracy: 0.6271 - loss: 0.7004
Epoch 2: val_accuracy improved from 0.69333 to 0.73778, saving model to model_224_best_model.keras
50/50 ━━━━━━━━━━━━━━━━━━━━ 24s 490ms/step - AUC: 0.7160 - Precision: 0.7070 - Recall: 0.6188 - accuracy: 0.6277 - loss: 0.6999 - val_AUC: 0.8360 - val_Precision: 0.7470 - val_Recall: 0.6200 - val_accuracy: 0.7378 - va

In [32]:

# Comparison of results with rounded metrics
print(f"\nComparison of models:")

print(f"Model with 224x224 resize: Accuracy={round(results_224[1], 3)}, AUC={round(results_224[2], 3)}, Precision={round(results_224[3], 3)}, Recall={round(results_224[4], 3)}, Time={time_64:.2f} seconds")
print(f"Model with 128x128 resize: Accuracy={round(results_128[1], 3)}, AUC={round(results_128[2], 3)}, Precision={round(results_128[3], 3)}, Recall={round(results_128[4], 3)}, Time={time_128:.2f} seconds")
print(f"Model with 64x64 resize: Accuracy={round(results_64[1], 3)}, AUC={round(results_64[2], 3)}, Precision={round(results_64[3], 3)}, Recall={round(results_64[4], 3)}, Time={time_64:.2f} seconds")
print("\n")

print(f"Model with 243x243 resize: Accuracy={round(results_243[1], 3)}, AUC={round(results_243[2], 3)}, Precision={round(results_243[3], 3)}, Recall={round(results_243[4], 3)}, Time={time_243:.2f} seconds")
print(f"Model with 162x162 resize: Accuracy={round(results_162[1], 3)}, AUC={round(results_162[2], 3)}, Precision={round(results_162[3], 3)}, Recall={round(results_162[4], 3)}, Time={time_162:.2f} seconds")
print(f"Model with 81x81 resize: Accuracy={round(results_81[1],3)}, AUC={round(results_81[2],3)}, Precision={round(results_81[3],3)}, Recall={round(results_81[4],3)}, Time={time_81:.2f} seconds")




Comparison of models:
Model with 224x224 resize: Accuracy=0.771, AUC=0.893, Precision=0.904, Recall=0.543, Time=69.70 seconds
Model with 128x128 resize: Accuracy=0.786, AUC=0.866, Precision=0.798, Recall=0.692, Time=82.99 seconds
Model with 64x64 resize: Accuracy=0.696, AUC=0.799, Precision=0.636, Recall=0.735, Time=69.70 seconds


Model with 243x243 resize: Accuracy=0.782, AUC=0.868, Precision=0.821, Recall=0.652, Time=300.33 seconds
Model with 162x162 resize: Accuracy=0.804, AUC=0.912, Precision=0.881, Recall=0.647, Time=165.18 seconds
Model with 81x81 resize: Accuracy=0.739, AUC=0.825, Precision=0.691, Recall=0.745, Time=54.60 seconds


In [14]:
model.evaluate(train_generator)

ValueError: You must call `compile()` before using the model.